In [1]:
import numpy as np
from scipy.integrate import RK45,solve_ivp
from ODE_potentials import VanDerPolePotential,LotkiVolterraPotential
from ODE_samplers import MALA_ODE,ULA_ODE,grad_ascent_ODE,run_eval_test,set_function
from multiprocessing import Pool
import multiprocessing
from zv_cv import Eval_ZVCV

from baselines import construct_ESVM_kernel
from optimize import optimize_parallel_new
from utils import *

Parameters for van-der-Pole and Lotki-Volterra examples: 

In [2]:
typ = 'LV' #'LV' for Lotki-Volterra, 'VdP' for Van-der-Pole
method = {"sampler":"MALA"} #switch between ULA and MALA
f_type = "sum_comps"

if typ == 'VdP':
    #true parameter value
    theta_star = 1.0
    #initial coordiante and speed
    y0 = np.array([0.0,2.0],dtype=float)
    #error of measurements
    sigma = 0.5
    #prior variance
    sigma_prior = 0.5
elif typ == 'LV':
    theta_star = np.array([0.6,0.025,0.8,0.025],dtype = float)
    #initial number of victims and predators
    y0 = np.array([30.0,4.0],dtype=float)
    #setting prior parameters
    sigma_prior = np.array([0.5,0.05,0.5,0.05],dtype = float)
    mu_prior = np.array([1.0,0.05,1.0,0.05],dtype=float)
    #measurements error
    sigma = np.array([0.25,0.25])

Timestaps

In [3]:
#initial and last time moments
t0 = 0
t_bound = 10
N_steps = 10
#moments of observations
t_moments = np.linspace(t0,t_bound,N_steps+1)
print(t_moments)

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]


Creating potentials

In [4]:
if typ == 'VdP':
    Cur_pot = VanDerPolePotential(sigma,sigma_prior,t_moments,theta_star,y0,t0,t_bound)
elif typ == 'LV':
    Cur_pot = LotkiVolterraPotential(sigma,mu_prior,sigma_prior,t_moments,theta_star,y0,t0,t_bound)

system solved
[[ 37.52358801   5.44451059]
 [ 49.31015054   4.44056815]
 [ 68.02780239   7.49382876]
 [ 96.31994597  69.0224216 ]
 [ 33.96485552 119.57915384]
 [ 11.19476367  55.29111344]
 [  7.28510513  25.46629586]
 [  3.95843583  11.12542133]
 [ 12.06748946   3.71380176]
 [ 12.38341447   6.01556616]
 [ 18.28774199   3.71351383]]


Sampling (currently with MALA)

In [5]:
r_seed = 666
#burn-in period
N_burn = 1*10**3
#Train size
N_train = 1*10**4
#Test size
N_test = 1*10**4
#number of test trajectories
n_traj_test = 20
if typ == 'VdP':
    #dimension
    d = 1
    #step size
    step = 1e-3
elif typ == 'LV':
    #dimension
    d = 4
    #step size
    step = 5e-6

In [6]:
bn = 20
W_test = construct_ESVM_kernel(N_test,bn)

### Construct kernels and sample

In [7]:
# Construct kernel
#W_train_spec = construct_ESVM_kernel(N_train)
#W_test_spec = construct_ESVM_kernel(N_test)

In [8]:
if typ == 'VdP':
    params_prior = {"sigma":sigma_prior}
elif typ == 'LV':
    params_prior = {"mu":mu_prior,"sigma":sigma_prior}

### Compute starting point (maximum likelihood)

In [9]:
N_steps_ascent = 10000
traj,traj_grad = grad_ascent_ODE(1453,Cur_pot,step,params_prior,N_steps_ascent,d,typ)
theta_mle = traj[-1,:]
print("mle for parameters: ",theta_mle)
Cur_pot.set_mle(theta_mle)

mle for parameters:  [0.6467501  0.02390763 0.81164578 0.02461639]


In [10]:
print(traj_grad[-10:,:])

[[-2.24829044e-11 -3.70759423e-10 -8.35420622e-12 -7.99621702e-10]
 [ 2.17386109e-11  2.15825580e-10  4.08562073e-13  6.65506761e-10]
 [-8.64153193e-12  3.10418358e-11  4.99866815e-12 -2.03080219e-10]
 [-2.23390195e-11  1.58593139e-11  2.17879048e-11 -6.36415365e-10]
 [ 6.86872781e-12  8.06004152e-11  1.55591096e-11  1.29166011e-10]
 [ 3.61310981e-12 -2.37800890e-11  3.34887673e-12  5.84261528e-11]
 [ 2.38555842e-11  1.76767045e-10 -1.66844316e-12  7.31555261e-10]
 [-2.60649280e-11 -3.28492789e-10 -2.35722553e-12 -8.47359516e-10]
 [ 1.46118673e-11 -3.61382035e-11 -7.55084884e-12  3.53569618e-10]
 [-4.33431069e-13 -1.08492770e-10 -2.29150032e-12 -8.99920138e-11]]


### Setting function 

In [ ]:
inds_arr = np.array([0])
params = {"ind":0}
t_moments = None
r_seed = 777
n_traj = 24
traj = []
traj_grad = []
#genertae data
nbcores = multiprocessing.cpu_count()
trav = Pool(nbcores)
res = trav.starmap(MALA_ODE, [(r_seed+i,Cur_pot, step, params_prior, N_burn, N_train, d, typ) for i in range (n_traj)])
trav.close()
for i in range(len(res)):
    traj.append(res[i][0])
    traj_grad.append(res[i][1])
    print("accepted = ",res[i][2])
traj = np.asarray(traj)
traj_grad = np.asarray(traj_grad)
traj_grad = (-1)*traj_grad

### Apply control variates

In [ ]:
#f_type = "sum"
f_type = "sum"
params = {"ind":3}
nbcores = multiprocessing.cpu_count()
trav = Pool(nbcores)
res = trav.starmap(Eval_ZVCV, [(traj[i,:,:],traj_grad[i,:,:],f_type,params,W_test) for i in range (n_traj)])
trav.close()

In [ ]:
print(f_type)

In [ ]:
res_arr = np.asarray(res)
print(res_arr.shape)

### Save results

In [ ]:
np.save("ode/mala_step_1e-5_obs_20_sum_15_04.npy",res_arr)
print("Average vr rates:")
print("ZV-1:",np.mean(res_arr[:,1,0]/res_arr[:,1,1]))
print("CV-1:",np.mean(res_arr[:,1,0]/res_arr[:,1,3]))
print("ZV-2:",np.mean(res_arr[:,1,0]/res_arr[:,1,2]))
print("CV-2:",np.mean(res_arr[:,1,0]/res_arr[:,1,4]))

### Plotting the results

In [ ]:
title = ""
labels = ['Vanilla\n MALA', 'MALA \nwith ZV-1', 'MALA \nwith CV-1']

In [ ]:
data = [res_arr[:,0,0],res_arr[:,0,1],res_arr[:,0,3]] 
boxplot_ind(data, title, labels)

In [ ]:
title = ""
labels = ['MALA \nwith ZV-1', 'MALA \nwith CV-1']

In [ ]:
data = [res_arr[:,0,1],res_arr[:,0,3]] 
boxplot_ind(data, title, labels)

In [ ]:
title = ""
labels = ['Vanilla\n MALA', 'MALA \nwith ZV-2', 'MALA \nwith CV-2']

In [ ]:
data = [res_arr[:,0,0],res_arr[:,0,2],res_arr[:,0,4]] 
boxplot_ind(data, title, labels)

In [ ]:
title = ""
labels = ['MALA \nwith ZV-2', 'MALA \nwith CV-2']

In [ ]:
data = [res_arr[:,0,2],res_arr[:,0,4]] 
boxplot_ind(data, title, labels)